# Sec 5.7: Calibrations of the Model
## and Appendix E: Fluctuating social value of nonwork

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.tsa.filters.hp_filter as hp

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.style.use('fivethirtyeight')

In [ ]:
import sys
sys.path.insert(0, '../')
import bug

## Read the data
Here, we read from the [excel file](https://github.com/pascalmichaillat/unemployment-gap/blob/main/code/data.xlsx) provided with the unemployment-gap matlab package.

The goal of this notebook is to re-create analysis and some figures from the Unemployment Gap paper, so that we can verify we are getting the *same* outputs. (*Sameness* allowing for some small differences between the two language implementations)

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Recession dates', header=1, 
                   usecols=['Peak month', 'Trough month'],).drop([0]).reset_index() 
starts =  pd.to_datetime(df['Peak month'])
ends = pd.to_datetime(df['Trough month'])

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment rate (percent)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_rate = pd.Series(data=df['Unemployment rate (percent)'].values,
                       index=dates, name='unempl_rate')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Vacancy rate (thousands)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
vac_rate_proxy = pd.Series(data=df['Vacancy rate (thousands)'].values,
                       index=dates, name='vacancy_rate_proxy')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Vacancy level (thousands)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
vacancy_level = pd.Series(data=df['Vacancy level (thousands)'].values,
                       index=dates, name='vacancy_level')

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Labor force level (thousands of persons)', 'Year', 'Month'],)
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
labor_level = pd.Series(data=df['Labor force level (thousands of persons)'].values,
                       index=dates, name='labor_force_level')

In [ ]:
vacancy_rate_2001 = vacancy_level/labor_level
vacancy_rate_splice = pd.concat([vac_rate_proxy.loc[:'2000-12'], vacancy_rate_2001.loc['2001-01':]*100])

In [ ]:
u_q = unempl_rate.resample('Q').mean()/100
u_q = u_q.loc[:u_q.last_valid_index()]

v_q = vacancy_rate_splice.resample('Q').mean()/100
v_q = v_q.loc[:v_q.last_valid_index()]

## Sec 5.7
### Matching elasticity
Recall, `bug.compute_beveridge_elasticity()` function is expecting the **log** of the unemployment and vacancy rates series. 

In [ ]:
bev_e, _ = bug.compute_beveridge_elasticity(np.log(u_q), np.log(v_q) )

In [ ]:
bev_e.head()

In [ ]:
bug.plot_beveridge_elasticity_series(bev_e, recession_dates=[starts, ends],draw_legend=True)
plt.ylim(0, 1.5)

### Effect of Bev elasticity

In [ ]:
# social value of non-work
zeta= 0.26
zeta_range = (0.03, 0.49)

In [ ]:
# recruiting costs
kappa=0.92
kappa_range = (0.61, 1.23)

In [ ]:
eff_unempl = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta, kappa=kappa)
eff_unempl_ub = bug.compute_efficient_unemployment(u_q, v_q, bev_e['UB'], zeta=zeta, 
                                                   kappa=kappa)
eff_unempl_lb = bug.compute_efficient_unemployment(u_q, v_q, bev_e['LB'], zeta=zeta, 
                                                   kappa=kappa)


# FIGURE 8A
## Beveridge elasticity 95% CI

In [ ]:
ax = u_q.plot(color='navy', linewidth=2, figsize=(10, 7), label='Actual')
eff_unempl.plot(ax=ax,color='magenta', linewidth=2,label='Efficient')
eff_unempl_ub.plot(ax=ax,color='magenta', linestyle='dotted', linewidth=2, label='_nolegend_')
eff_unempl_lb.plot(ax=ax,color='magenta',linestyle='dotted',  linewidth=2, label='_nolegend_')
plt.fill_between(eff_unempl_lb.index, eff_unempl_lb, eff_unempl_ub, color='magenta', alpha=.2)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, 
                augment_legend=True, legend_loc=4)

plt.ylim(0, .12)
plt.ylabel('Unemployment rate', fontsize=12)
plt.title('Unemployment', fontsize=14)
plt.suptitle('With Beveridge elasticity 95% CI')

# FIGURE 8B
### Effect of zeta Value of non-work

In [ ]:
eff_unempl = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta, kappa=kappa)
eff_unempl_ub = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta_range[1], 
                                                   kappa=kappa)
eff_unempl_lb = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta_range[0], 
                                                   kappa=kappa)

In [ ]:
ax = u_q.plot(color='navy', linewidth=2, figsize=(10, 7), label='Actual')
eff_unempl.plot(ax=ax,color='magenta', linewidth=2,label='Efficient')
eff_unempl_ub.plot(ax=ax,color='magenta', linestyle='dotted', linewidth=2, label='_nolegend_')
eff_unempl_lb.plot(ax=ax,color='magenta',linestyle='dotted',  linewidth=2, label='_nolegend_')
plt.fill_between(eff_unempl_lb.index, eff_unempl_lb, eff_unempl_ub, color='magenta', alpha=.2)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=4)

plt.ylim(0, .12)
plt.ylabel('Unemployment rate', fontsize=12)
plt.title('Unemployment', fontsize=14)
plt.suptitle('With zeta value non-work in [0.03, 0.49]')

# FIGURE 8C
### Effect of kappa recruiting cost

In [ ]:
eff_unempl = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta, kappa=kappa)
eff_unempl_ub = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta, 
                                                   kappa=kappa_range[1])
eff_unempl_lb = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta, 
                                                   kappa=kappa_range[0])

In [ ]:
ax = u_q.plot(color='navy', linewidth=2, figsize=(10, 7), label='Actual')
eff_unempl.plot(ax=ax,color='magenta', linewidth=2,label='Efficient')
eff_unempl_ub.plot(ax=ax,color='magenta', linestyle='dotted', linewidth=2, label='_nolegend_')
eff_unempl_lb.plot(ax=ax,color='magenta',linestyle='dotted',  linewidth=2, label='_nolegend_')
plt.fill_between(eff_unempl_lb.index, eff_unempl_lb, eff_unempl_ub, color='magenta', alpha=.2)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=4)

plt.ylim(0, .12)
plt.ylabel('Unemployment rate', fontsize=12)
plt.title('Unemployment', fontsize=14)
plt.suptitle('With kappa recruiting cost in [0.61, 1.23]')

# FIGURE 9A
### inverse-optimum Beveridge elasticity
Compute `bug.compute_beveridge_inverse()` with:
  * theta, required
  * zeta, optional, default zeta=0.26
  * kappa, optional, default kappa=0.92

In [ ]:
theta = v_q/u_q
e_star = bug.compute_beveridge_inverse(theta)

In [ ]:
ax = e_star.plot(color='darkred', linewidth=2, figsize=(10, 7), label='Inverse Optimum')
bev_e['E'].plot(ax=ax,color='blueviolet', linewidth=2,label='Bev Elasticity')
bev_e['UB'].plot(ax=ax,color='blueviolet', linestyle='dotted', linewidth=2, 
                 label='_nolegend_')
bev_e['LB'].plot(ax=ax,color='blueviolet', linestyle='dotted', linewidth=2, 
                 label='_nolegend_')
plt.fill_between(bev_e['UB'].index, bev_e['LB'], bev_e['UB'], color='blueviolet', alpha=.3)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=2)

plt.ylabel('Beveridge elasticity', fontsize=12)
plt.title('Beveridge elasticity 95% CI', fontsize=14)

# FIGURE 9B
### inverse-optimum zeta value of non-work
Compute `bug.compute_nonwork_inverse()` with:
  * theta, required
  * Beveridge elasticity, requried
  * kappa, optional, default kappa=0.92

In [ ]:
zeta_star = bug.compute_nonwork_inverse(theta, bev_e['E'], kappa)

In [ ]:
ax = zeta_star.plot(color='darkred', linewidth=2, figsize=(10, 7), label='Inverse Optimum')
plt.axhline(zeta_range[0],color='blueviolet', linestyle='dotted', linewidth=2, 
            label='_nolegend_')
plt.axhline(zeta_range[1],color='blueviolet', linestyle='dotted', linewidth=2, 
            label='_nolegend_')
plt.axhline(zeta,color='blueviolet', linewidth=2, label='value non-work')

plt.fill_between(zeta_star.index, zeta_range[0], zeta_range[1], color='blueviolet', alpha=.3)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=4)

plt.ylim(-.5, 1)
plt.ylabel('Zeta', fontsize=12)
plt.title('Zeta value of Non-work', fontsize=14)

# FIGURE 9C
### inverse-optimum kappa
Compute `bug.compute_recruiting_inverse()` with:
  * theta, required
  * Beveridge elasticity, requried
  * zeta, optional, default zeta=0.26

In [ ]:
kappa_star = bug.compute_recruiting_inverse(theta, bev_e['E'], zeta)

In [ ]:
ax = kappa_star.plot(color='darkred', linewidth=2, figsize=(10, 7), label='Inverse Optimum')
plt.axhline(kappa_range[0],color='blueviolet', linestyle='dotted', linewidth=2, 
            label='_nolegend_')
plt.axhline(kappa_range[1],color='blueviolet', linestyle='dotted', linewidth=2, 
            label='_nolegend_')
plt.axhline(kappa,color='blueviolet', linewidth=2, label='Recruiting Cost')

plt.fill_between(kappa_star.index, kappa_range[0], kappa_range[1], color='blueviolet', 
                 alpha=.3)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=2)

plt.ylim(0, 6)
plt.ylabel('kappa', fontsize=12)
plt.title('Kappa Recruiting Cost', fontsize=14)

# FIGURE 10
### with zeta = 0.96

In [ ]:
zeta_HM = 0.96

In [ ]:
u_HM = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta=zeta_HM)

In [ ]:
ax = u_HM.plot(color='magenta', linewidth=2, figsize=(9, 6), label='Efficent')
u_q.plot(color='navy', linewidth=2,  label='Actual')

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=2)

plt.ylim(0,.32)
plt.ylabel('Unemployment', fontsize=12)
plt.title('Unemployment', fontsize=14)
plt.suptitle('With zeta = 0.96')

# Appendix E
### Fluctuating social value of nonwork
Read in data on labor productivity

In [ ]:
labor_p = pd.read_excel('../../code/data.xlsx', sheet_name='Quarterly data', header=1, 
                           usecols=['Real output per person (index)', 'Year', 'Quarter'],)
# set the index 
labor_p['date'] = labor_p['Year'].astype(str) +'-Q' + labor_p['Quarter'].astype(str)
labor_p['date'] = pd.PeriodIndex(labor_p['date'], freq='Q')
labor_p = labor_p.set_index('date')
labor_p.drop(columns=['Year', 'Quarter'], inplace=True)
labor_p.rename(columns={'Real output per person (index)': 'Labor productivity'}, inplace=True)

# FIGURE A8a
### Labor productivity in the United States, 1951–2019
#### run a high-pass filter

In [ ]:
cycle, trend = hp.hpfilter(np.log(labor_p), 1600)

In [ ]:
ax = labor_p.plot(linewidth=3, figsize=(9, 6),)
np.exp(trend).plot(linewidth=1.5, color='red')

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=2)

plt.ylim(0,120)
plt.ylabel('Productivity Index', fontsize=12)
plt.title('Productivity', fontsize=14)

# FIGURE A8b

In [ ]:
ax = np.exp(cycle).plot(linewidth=2, color='purple',figsize=(9, 6),)

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=2)
        
plt.ylim(.94,1.06)
plt.ylabel('Detrended Productivity Index', fontsize=12)
plt.title('Detrended Productivity', fontsize=14)

# FIGURE A9
### US efficient unemployment rate with fluctuating social value of nonwork

In [ ]:
u_fluct = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'], zeta= zeta/np.exp(cycle))

In [ ]:
u_const = bug.compute_efficient_unemployment(u_q, v_q, bev_e['E'],)

In [ ]:
ax = u_const.plot(linewidth=1.5, color='magenta',figsize=(9, 6),label='Const social value')
u_fluct.plot(color='darkblue', linewidth=3, linestyle='dotted', label='Fluct social value')

bug.format_plot(ax, recession_dates=[starts, ends], xgrid=True, augment_legend=True, 
                legend_loc=4)

plt.ylim(0,.06)
plt.ylabel('Unemployment', fontsize=12)
plt.title('Unemployment', fontsize=14)